# Image_Heatmap_TensorFlow

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [資料處理參數設定](#3)
1. [資料處理](#4)
    -  [載入模型](#4.1)
    -  [定義方法](#4.2)
    -  [批次圖片處理](#4.3)
        -  [生成圖片資料集](#4.3.1)
        -  [列出原始圖片](#4.3.2)
        -  [生成熱視圖](#4.3.3)
        -  [疊加圖片](#4.3.4)
    -  [單圖片處理](#4.4)
        -  [列出原始圖片](#4.4.1)
        -  [生成熱視圖](#4.4.2)
        -  [疊加圖片](#4.4.3)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# !pip install -q efficientnet
# import efficientnet.tfkeras as efn

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons.optimizers as addons_optimizers

In [ ]:
# 資料處理套件
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# # tensorflow深度學習模組套件
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
# 查看設備
print(device_lib.list_local_devices())

In [ ]:
# 查看tensorflow版本
print(tf.__version__)

# 查看圖像通道位置
print(K.image_data_format())

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = True

# (Boolean)是否為 Colab
COLAB = False

# (String)CPU/GPU/TPU
DEVICE = "GPU"


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    
# (String)資料根路徑
# DATA_ROOT_PATH = PATH+r'datasets/AI_CUP_2020_AIMango_Grade_Classification/' 
DATA_ROOT_PATH = PATH+r'datasets/faces_glintasia_112x112_folders/' 

# (String)訓練資料路徑
# TRAIN_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Test_Cropped'
TRAIN_DATA_PATH = DATA_ROOT_PATH

# (String)訓練CSV路徑，如為None則不讀CSV檔
TRAIN_CSV_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/dev.csv'

# (String)測試單張圖片檔案路徑
# TRAIN_FILE_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Test_Cropped'
TRAIN_FILE_DATA_PATH = DATA_ROOT_PATH+r'2'
    
# (String)測試單張圖片檔名
# IMAGE_PATH =  TRAIN_FILE_DATA_PATH + '/00001.jpg'
IMAGE_PATH =  TRAIN_FILE_DATA_PATH + '/34.jpg'

# (String)儲存單張熱視圖的路徑檔名
SINGLE_HEATMAP_PATH = "single_heatmap.png"

# (String)儲存多張熱視圖的路徑檔名
BATCH_HEATMAP_PATH = "batch_heatmap.png"

# (String)模型名稱(使用哪個模型)
# MODEL_NAME = 'efficientnetb7'
# MODEL_NAME = 'se_mobile_facenet_glintasia_1000'
MODEL_NAME = 'se_mobile_facenet_glintasia_2000'

# (String)專案名稱
# PROJECT_NAME = 'AI_CUP_2020_AIMango_Grade_Classification'
PROJECT_NAME = 'Face_Recognition'

# (String)模型的專案路徑
PROJECT_PATH = PATH + r'models/'+PROJECT_NAME

# (String)讀取模型的儲存路徑 
LOAD_MODEL_PATH = PROJECT_PATH + r'/' + MODEL_NAME + '.h5'

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if DEVICE != "CPU":
    !nvidia-smi

In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
# 動態申請顯存
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
if os.path.isfile(TRAIN_CSV_PATH):
    LOAD_CSV = True
else:
    LOAD_CSV = False

# 3. 資料處理參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)圖片尺寸
# IMAGE_SIZE = 224
IMAGE_SIZE = 112

# (String)CSV圖片檔名欄位(不包含路徑)
IMAGE_NAME = 'image_id'

# (String)CSV圖片檔名欄位(包含路徑)
IMAGE_NAME_ROOT = 'image'

# (String)CSV標籤欄位
LABEL_NAME = 'grade'

# (String)CSV標籤欄位類型
LABEL_NAME_TYPE = 'string'

# (Boolean)是否顯示標籤
DISPLAY_LABEL = True
    
# (Boolean)是否印出完整模型
MODEL_PRINT = False

# (Boolean)是否印出模型網路層
MODEL_LAYERS_PRINT = False

# (Boolean)是否生成批次熱視圖
GENERATE_BATCH_HEATMAP = True

# (Boolean)是否儲存批次熱視圖
SAVE_BATCH_HEATMAP = False

# (Boolean)是否生成單張熱視圖
GENERATE_SINGLE_HEATMAP = True

# (Boolean)是否儲存單張熱視圖
SAVE_SINGLE_HEATMAP = False

# (Float)熱視強度
INTENSITY = 0.4

# (String)熱視圖Last Conv層Layer Name
# LAST_CONV_LAYER_NAME = "top_conv"
LAST_CONV_LAYER_NAME = "conv2d_61"

# (String List)熱視圖Conv層之後的Classifier Layer Names
# CLASSIFIER_LAYER_NAMES = [
#     "top_bn",
#     "top_activation",
#     "averagepooling2d_head",
#     "flatten_head",
#     "dense_head",
#     "dropout_head",
#     "predictions_head"
# ]
CLASSIFIER_LAYER_NAMES = [
    "batch_normalization_47",
    "p_re_lu_47",
    "depthwise_conv2d_15",
    "batch_normalization_48",
    "conv2d_62",
    "batch_normalization_49",
    "p_re_lu_48",
    "dropout",
    "flatten",
    "dense",
    "embedding",
    "softmax"
]


'''資料集製作參數設定'''

#  (String)預設：'rgb'，圖像是否轉換為 1 個或 3 個顏色通道
COLOR_MODE = 'rgb'

#  (String)預設：'categorical'，決定返回標籤數組的類型："categorical" 將是 2D one-hot 編碼標籤
CLASS_MODE = 'categorical'

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

# (Float)驗證集佔訓練集的比率，FOLDS>1則不啟用
DATA_SPLIT = 0.2

# (Int)每批訓練的尺寸
BATCH_SIZE = 32


''''圖表參數設定'''

# (Int)原始圖片列表寬度
ORINAL_PLOT_FIGSIZE_W = 10

# (Int)原始圖片列表高度
ORINAL_PLOT_FIGSIZE_H = 10

# (Int)熱視圖列表寬度
HEATMAP_PLOT_FIGSIZE_W = 10

# (Int)熱視圖列表高度
HEATMAP_PLOT_FIGSIZE_H = 10

# (Int)熱視圖列表標題字型大小
HEATMAP_PLOT_TITLE_FONTSIZE = 15

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(SEED)

# 4. 資料處理<a class="anchor" id="4"></a>
https://keras.io/examples/vision/grad_cam/

[Back to Table of Contents](#0)

## 4.1 載入模型 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
print("Reading the pre-trained model... ")
model = keras.models.load_model(LOAD_MODEL_PATH)
print("Reading done. ")

if MODEL_PRINT:
    model.summary()
if MODEL_LAYERS_PRINT:
    for layer in model.layers:
        print(layer.name)

## 4.2 定義方法 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size IMAGE_SIZExIMAGE_SIZE
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (IMAGE_SIZE, IMAGE_SIZE, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, IMAGE_SIZE, IMAGE_SIZE, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(LAST_CONV_LAYER_NAME)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in CLASSIFIER_LAYER_NAMES:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap, top_pred_index.numpy()

In [ ]:
def superimposed_img(img, heatmap, intensity, single):
    if single:
        # We load the original image
        img = keras.preprocessing.image.load_img(img)
        img = keras.preprocessing.image.img_to_array(img)

    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * intensity + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    return superimposed_img

## 4.3 批次圖片處理 <a class="anchor" id="4.3"></a>
[Back to Table of Contents](#0)

### 4.3.1 生成圖片資料集 <a class="anchor" id="4.3.1"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    print('Reading data...')

    # 讀取訓練資料集CSV檔
    train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")

    print('Reading data completed')

In [ ]:
if LOAD_CSV:
    label_list = []
    train_list = []
    for i in range(train_csv.shape[0]):
        train_list.append(TRAIN_DATA_PATH + '/' + train_csv[IMAGE_NAME].iloc[i])
        label_list.append(train_csv[LABEL_NAME].iloc[i])
    df_train = pd.DataFrame(train_list)
    df_train.columns = [IMAGE_NAME_ROOT]
    df_train[LABEL_NAME] = label_list

In [ ]:
if LOAD_CSV:
    X_train, X_val, y_train, y_val = train_test_split(df_train[IMAGE_NAME_ROOT],df_train[LABEL_NAME], test_size = DATA_SPLIT, random_state = SEED)

    validation_data = pd.DataFrame(X_val)
    validation_data.columns = [IMAGE_NAME_ROOT]
    validation_data[LABEL_NAME] = y_val
    
    validation_data[LABEL_NAME] = validation_data[LABEL_NAME].astype(LABEL_NAME_TYPE)

    val_datagen = ImageDataGenerator(rescale = 1./255)
    
    validation_generator = val_datagen.flow_from_dataframe(
         validation_data,
         x_col = IMAGE_NAME_ROOT,
         y_col = LABEL_NAME,
         target_size = (IMAGE_SIZE, IMAGE_SIZE),
         batch_size = BATCH_SIZE,
         shuffle = False,
         seed = SEED,
         color_mode = COLOR_MODE,
         class_mode = CLASS_MODE)

In [ ]:
if not LOAD_CSV:
    validation_datagen = ImageDataGenerator(rescale = 1. / 255,
                                       validation_split = DATA_SPLIT)
    
    validation_generator = validation_datagen.flow_from_directory(
                TRAIN_DATA_PATH, 
                target_size = (IMAGE_SIZE, IMAGE_SIZE), 
                batch_size = BATCH_SIZE, 
                shuffle = False, 
                seed = SEED, 
                color_mode = COLOR_MODE, 
                class_mode = CLASS_MODE, 
                subset = 'validation')

In [ ]:
# 利用資料集找出分類項目
CLASSES_LIST = []
validation_dict = validation_generator.class_indices
for key, value in validation_dict.items():
    CLASSES_LIST.append(key)

In [ ]:
if  GENERATE_BATCH_HEATMAP:
    sample_data = validation_generator.__getitem__(1)[0] 
    sample_label = validation_generator.__getitem__(1)[1] 

### 4.3.2 列出原始圖片 <a class="anchor" id="4.3.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if  GENERATE_BATCH_HEATMAP:
    plt.figure(figsize=(ORINAL_PLOT_FIGSIZE_W,ORINAL_PLOT_FIGSIZE_H))
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.axis('off')
        plt.imshow(sample_data[i])

        if DISPLAY_LABEL:
            plt.title(CLASSES_LIST[np.argmax(sample_label[i])])

### 4.3.3 生成熱視圖 <a class="anchor" id="4.3.3"></a>
[Back to Table of Contents](#0)

In [ ]:
if  GENERATE_BATCH_HEATMAP:
    plt.figure(figsize=(HEATMAP_PLOT_FIGSIZE_W,HEATMAP_PLOT_FIGSIZE_H))
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.axis('off')
        heatmap, top_index = make_gradcam_heatmap(np.expand_dims(sample_data[i], axis=0), model, LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAMES)
        plt.imshow(heatmap)

        if DISPLAY_LABEL:
            plt.title(CLASSES_LIST[np.argmax(sample_label[i])] + " pred as: " + CLASSES_LIST[top_index], fontsize=HEATMAP_PLOT_TITLE_FONTSIZE)

### 4.3.4 疊加圖片 <a class="anchor" id="4.3.4"></a>
[Back to Table of Contents](#0)

In [ ]:
if  GENERATE_BATCH_HEATMAP:
    plt.figure(figsize=(HEATMAP_PLOT_FIGSIZE_W,HEATMAP_PLOT_FIGSIZE_H))
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.axis('off')
        heatmap, top_index = make_gradcam_heatmap(np.expand_dims(sample_data[i], axis=0), model, LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAMES)
        img = np.uint8(255 * sample_data[i])
        s_img = superimposed_img(img, heatmap, INTENSITY, False)
        plt.imshow(s_img)

        if DISPLAY_LABEL:
            plt.title(CLASSES_LIST[np.argmax(sample_label[i])] + " pred as: " + CLASSES_LIST[top_index], fontsize=HEATMAP_PLOT_TITLE_FONTSIZE)

    if SAVE_BATCH_HEATMAP:
        plt.savefig(BATCH_HEATMAP_PATH)

## 4.4 單圖片處理 <a class="anchor" id="4.4"></a>
[Back to Table of Contents](#0)

### 4.4.1 列出原始圖片 <a class="anchor" id="4.4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
if GENERATE_SINGLE_HEATMAP:
    # 圖片轉成陣列格式
    img_array = get_img_array(IMAGE_PATH, size=(IMAGE_SIZE,IMAGE_SIZE))

    # 對圖片進行編碼
    img_array = imagenet_utils.preprocess_input(img_array, mode = 'tf')

    # Imshow Image
    plt.imshow(plt.imread(IMAGE_PATH))
    plt.axis('off') # 不顯示座標軸
    plt.show()

### 4.4.2 生成熱視圖 <a class="anchor" id="4.4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if GENERATE_SINGLE_HEATMAP:
    heatmap, top_index = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAMES)

    if DISPLAY_LABEL:
        print("predicted as", CLASSES_LIST[top_index])

    # Imshow Heatmap
    plt.imshow(heatmap)
    plt.axis('off') # 不顯示座標軸
    plt.show()

### 4.4.3 疊加圖片 <a class="anchor" id="4.4.3"></a>
[Back to Table of Contents](#0)

In [ ]:
if GENERATE_SINGLE_HEATMAP:
    s_img = superimposed_img(IMAGE_PATH, heatmap, INTENSITY, True)

    # Imshow Grad CAM
    plt.imshow(s_img)
    plt.axis('off') # 不顯示座標軸
    if SAVE_SINGLE_HEATMAP:
        plt.savefig(SINGLE_HEATMAP_PATH)
    plt.show()